# Part A

- **Love**: Follow and maintain optimal distance
- **Fear**: Retreat from detected humans
- **Aggressive**: Relentless pursuit
- **Curious**: Cautious investigation with circling

Keyboard controls:
- 'l' = Love behavior
- 'f' = Fear behavior  
- 'a' = Aggressive behavior
- 'c' = Curious behavior
- 's' = Stop

In [1]:
!wget https://storage.googleapis.com/mediapipe-models/gesture_recognizer/gesture_recognizer/float16/latest/gesture_recognizer.task

--2025-12-05 09:49:17--  https://storage.googleapis.com/mediapipe-models/gesture_recognizer/gesture_recognizer/float16/latest/gesture_recognizer.task
Resolving storage.googleapis.com (storage.googleapis.com)... 2a00:1450:4009:c0b::cf, 2a00:1450:4009:c17::cf, 2a00:1450:4009:c13::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2a00:1450:4009:c0b::cf|:443... connected.
200 OKequest sent, awaiting response... 
Length: 8373440 (8.0M) [application/octet-stream]
Saving to: ‘gesture_recognizer.task.10’

gesture_recognizer. 100%[===================>]   7.99M  11.7MB/s    in 0.7s    

2025-12-05 09:49:17 (11.7 MB/s) - ‘gesture_recognizer.task.10’ saved [8373440/8373440]



**Step 1: Display Widgets**

Standard widget setup for displaying color and depth images side by side.

In [2]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create two widgets for displaying images
display_color = widgets.Image(format='jpeg', width='45%')
display_depth = widgets.Image(format='jpeg', width='45%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# convert numpy array to jpeg for display
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

**Step 2: Load YOLO Model**

load the TensorRT YOLO model for human detection.

In [3]:
from ultralytics import YOLO

model = YOLO("yolo11l_half.engine")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [4]:
!pip install mediapipe

Defaulting to user installation because normal site-packages is not writeable


**Step 3: Camera System Initialization**

In [5]:
import traitlets
import numpy as np
import pyzed.sl as sl
import threading
import motors
import time
import math
import random
from traitlets.config.configurable import SingletonConfigurable

# NEW: MediaPipe Gesture Recognition imports
import mediapipe as mp
from mediapipe.tasks.python import vision

# NEW: MediaPipe classes
BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = vision.GestureRecognizer
GestureRecognizerOptions = vision.GestureRecognizerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Initialize robot motor control
robot = motors.MotorsYukon(mecanum=False)

# Camera class with traitlets (SAME AS BEFORE)
class Camera(SingletonConfigurable):
    color_value = traitlets.Any()

    def __init__(self):
        super(Camera, self).__init__()

        # ZED camera initialization
        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False

        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
                
                self.color_value_BGRA = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())

    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

# Initialize and start camera
camera = Camera()
camera.start()

[2025-12-05 09:49:27 UTC][ZED][INFO] Logging level INFO
[2025-12-05 09:49:27 UTC][ZED][INFO] Logging level INFO
[2025-12-05 09:49:27 UTC][ZED][INFO] Logging level INFO
[2025-12-05 09:49:27 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-12-05 09:49:28 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-12-05 09:49:28 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-12-05 09:49:28 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-12-05 09:49:28 UTC][ZED][INFO] [Init]  Serial Number: S/N 36955685
Loading yolo11l_half.engine for TensorRT inference...
requirements: Ultralytics requirement ['numpy==1.23.5'] not found, attempting AutoUpdate...
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 10.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4

requirements: AutoUpdate success

**Step 4: Behavior Functions**

In [6]:
# Global variable for current behavior mode
current_behavior = 'stop'


def detect_human(frame):
    """
    Human detection and distance calculation
    
    From Tutorial 5:
    - YOLO detection for humans (class 0)
    - Bounding box extraction: result.boxes.xyxy[i]
    - Center calculation: (bbox[0] + bbox[2]) / 2 for x, (bbox[1] + bbox[3]) / 2 for y
    - Distance extraction: camera.depth_image[center_y, center_x]

    """

    results = model(frame, verbose=False)
    
    # confidence threshold for detections
    conf_threshold = 0.5
    human_detected = False
    min_distance = float('inf')
    closest_bbox = None
    
    # iterate through detections
    for result in results:
        for i in range(len(result.boxes.cls)):
            if result.boxes.cls[i] == 0:
                # Check confidence (Tutorial 5)
                if result.boxes.conf[i] > conf_threshold:
                    human_detected = True
                    # Get bounding box coordinates (Tutorial 5)
                    bbox = result.boxes.xyxy[i]
                    
                    # Calculate center of bounding box (Tutorial 5 method)
                    # bbox format: [x1, y1, x2, y2]
                    center_x = int((bbox[0] + bbox[2]) / 2)
                    center_y = int((bbox[1] + bbox[3]) / 2)
                    
                    # Get depth at human center (Tutorial 5)
                    # Depth image indexing: depth_image[row, col] or depth_image[y, x]
                    # Returns distance in millimeters
                    if (0 <= center_y < camera.height and 0 <= center_x < camera.width):
                        distance = camera.depth_image[center_y, center_x]
                        
                        # NaN means no depth data, negative or zero means invalid
                        if not np.isnan(distance) and distance > 0:
                            # Track closest human
                            if distance < min_distance:
                                min_distance = distance
                                closest_bbox = bbox
    
    return human_detected, min_distance, closest_bbox

def love_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """
    Love behavior - follow human and maintain comfortable distance
    
    Distance thresholds chosen based on testing:
    - < 450mm: Too close, back up
    - 450-600mm: Optimal distance, stop
    - 600-1200mm: Medium range, approach slowly
    - > 1200mm: Far away, approach faster
    
    """
    if human_detected and distance != float('inf'):
        # Draw dectection box(Tutorial 5 pattern)
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (255, 105, 180), 2)
        
         # Calculate if human is left/right of center (Tutorial 5 approach)
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2

        # Distance-based behavior (new thresholds for love behavior)
        if distance < 450:
            robot.backward(0.5)
            cv2.putText(depth_colormap, f'LOVE: Too close {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        elif distance >= 450 and distance <= 600:
            robot.stop()
            cv2.putText(depth_colormap, f'LOVE: Perfect {distance:.0f}mm <3', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
        elif distance > 600 and distance <= 1200:
            # Medium distance - turn to center, then approach
            # 50 pixel threshold to avoid constant adjustment
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.3) 
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.3) 
            else:
                robot.forward(0.3) 
            cv2.putText(depth_colormap, f'LOVE: Approaching {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2)
        else:  # distance > 1200
            # Far away - approach faster with turning
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.4)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.4)
            else:
                robot.forward(0.5)
            cv2.putText(depth_colormap, f'LOVE: Coming {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)
    else:
        # No human detected - stop and wait
        robot.stop()
        cv2.putText(depth_colormap, 'LOVE: Waiting...', 
                   (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)

# Global for fear like a mouse or rabbit
# escape zone
escape_start = None
escape_direction = None
escape_phase = None

def fear_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """
    Fear behavior - retreat from humans and runs away 
    
    Distance thresholds:
    - < 500mm: DANGER - fast retreat (0.6 speed)
    - < 900mm: CAUTION - medium retreat (0.4 speed)
    - < 1300mm: ALERT - slow retreat (0.2 speed)
    - >= 1300mm: SAFE - stop retreating

    robot will reach a certain once safw, turn 90 degree and run for 5 sec 
    """
    global escape_direction, escape_phase, escape_start

    if human_detected and distance != float('inf'):
        # Draw red warning box
        cv2.rectangle( frame, (int(bbox[0]), int(bbox[1])),
                      (int(bbox[2]), int(bbox[3])), (0, 0, 255), 2)

        # Graduated fear respone
        if distance < 500:
            # Very close fast retreat
            escape_phase = 'retreating'
            escape_start = None
            robot.backward(0.6)
            cv2.putText(depth_colormap, f'Fear: DANGER {distance:.0f}mm!',
                        (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        elif distance < 900:
            #Medium distance medium retract
            escape_phase = 'retreating'
            escape_start = None
            robot.backward(0.4)
            cv2.putText(depth_colormap, f'Fear: Caution {distance:.0f}mm',
                        (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2)
        elif distance < 1300:
            # Far but caution, slow retreat
            escape_phase = 'retreating'
            escape_start = None
            robot.backward(0.2)
            cv2.putText(depth_colormap, f'Fear: Alert {distance:.0f}mm',
                        (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        else:
            # safe distance reached
            if escape_phase == 'retreating':
                # will start to run away
                escape_phase = 'turning'
                escape_start = time.time()
                escape_direction = random.choice(['left', 'right'])
            
            if escape_phase == 'turning':
                # turn 90 degrees
                turn_duration = 1.0

                if time.time() - escape_start < turn_duration:
                    if escape_direction == 'left': # make turn
                        robot.left(0.5)
                        cv2.putText(depth_colormap, f'Fear : Turning left',
                                    (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                    else:
                        robot.right(0.5)
                        cv2.putText(depth_colormap, f'Fear : Turning right',
                                    (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                else:
                    # will start to run away
                    escape_phase = 'running'
                    escape_start = time.time()
            
            elif escape_phase == 'running':
                runaway = 5.0 # will run for 5 secs
                if time.time() - escape_start < runaway:
                    robot.forward(0.6)
                    remaining = runaway - (time.time() - escape_start)
                    cv2.putText(depth_colormap, f'Fear : running or going to hide {remaining:.1f}s',
                                    (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
                else: # will stop and restrat normal action
                    robot.stop()
                    escape_phase = None
                    escape_start = None
                    cv2.putText(depth_colormap, f'Fear : I am safe from target',
                                    (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            
            else: # robot is safe
                robot.stop()
                cv2.putText(depth_colormap, f'Fear : safe{distance:.0f}mm',
                                    (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    else: 
        # no threat but escape in progress
        if escape_phase == 'running':
            runaway = 5.0
            if time.time() - escape_start < runaway:
                # will keep runnning even if target is yet to be seen
                robot.forward(0.6)
                remaining = runaway - (time.time() - escape_start)
                cv2.putText(depth_colormap, f'Fear : running or going to hide {remaining:.1f}s',
                                    (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
            else: 
                # wont run
                robot.stop()
                escape_phase = None
                cv2.putText(depth_colormap, f'Fear : no threat',
                                    (2000, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255, 0), 2)
            
        elif escape_phase == 'turning':
            #will turn even target is not around
            turn_duration = 1.0
            if time.time() - escape_start < turn_duration:
                if escape_direction == 'left':
                    robot.left(0.5)
                else:
                    robot.right(0.5)
            else: # start running
                escape_phase = 'running'
                escape_start = time.time()

        else: 
            # No threat detected
            robot.stop()
            cv2.putText(depth_colormap, f'Fear: No threat ',
                        (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

# Global for aggression like a dog and a bull
dog_movement = { 'active':False, 
                'rush': 0, 
                'charging':None, 
                'barking':0} # similar why a dog barks at people and tire to attack

def aggressive_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """
    Aggressive behavior - relentlessly push toward humans without stopping
    - NEVER stops when target detected (shows more aggression)
    - Speed scales with distance (closer = more intense)
    - Continuously rams even at close range
    
    Speed scaling:
    - > 1000mm: 0.4 speed (chasing)
    - > 600mm: 0.6 speed (rushing)
    - > 300mm: 0.8 speed (engaging)
    - <= 300mm: 1.0 speed (destroy - keep ramming)
    """
    global dog_movement
    
    if human_detected and distance != float('inf'):
        # Draw orange box
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (0, 165, 255), 2)
        
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2

        # dog movement logic
        if 250 < distance <= 400:
            if not dog_movement['active']:
                dog_movement['active'] = True
                dog_movement['barking'] = 0
                dog_movement['charging'] = 'forward'
                dog_movement['rush'] = time.time()

                # 3 to laps of attacking
            if dog_movement['barking'] < 1:
                current_time = time.time()
                stopped = current_time - dog_movement['rush']

                    #go to attack forward like warning in a way
                if dog_movement['charging'] == 'forward':
                    if stopped < 0.5:
                        robot.forward(0.8)
                        cv2.putText(depth_colormap, 
                                        f'dog or bull: action {dog_movement["barking"]+1}/1 - {distance:.0f}mm',
                                        (80, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
                        print(f'dog or bull : forward movement {dog_movement["barking"]+1}/1', end='\r')
                    else: # will be go backward
                        dog_movement['charging'] = 'backward'
                        dog_movement['rush'] = current_time
                    #go back breiftly
                elif dog_movement['charging'] == 'backward':
                    if stopped < 0.5:
                            robot.backward(0.8)
                            cv2.putText(depth_colormap, 
                                        f'dog or bull: go back {dog_movement["barking"]+1}/1- {distance:.0f}mm',
                                        (80, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
                            print(f'dog or bull : back {dog_movement["barking"]+1}/1', end='\r')
                    else: # one cycle
                            dog_movement['barking']  += 1
                            dog_movement['charging'] = 'forward'
                            dog_movement['rush'] = current_time

                return
                
            else: # finsh the 1 cycle
                    dog_movement['active'] = False   

        if distance > 400 or distance <= 250:
            dog_movement['active'] = False
            dog_movement['barking'] = 0

        # Determine speed based on distance - faster when farther
        if distance > 1000:
            speed = 0.4
            intensity = "chasing"
            color = (255, 165, 0)  # orange
        elif distance > 600:
            speed = 0.6
            intensity = "chasing"
            color = (255, 69, 0)  # red-orange
        elif distance > 400:
            speed = 0.8
            intensity = "engage attack"
            color = (255, 0, 0)  # red
        elif distance > 250:
            speed = 0.7
            intensity = "attack"
            color = (255, 0, 0)  # red
        else:
            # Close range - KEEP PUSHING at maximum speed
            speed = 1.0
            intensity = "destroy"
            color = (139, 0, 0)  # dark-red
        
        
        # Always move forward or turn toward human - NEVER STOP
        if bbox_center_x < frame_center_x - 50:
            robot.left(0.5)
            cv2.putText(depth_colormap, f'AGGRESSIVE: {intensity} LEFT {distance:.0f}mm', 
                       (100, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        elif bbox_center_x > frame_center_x + 50:
            robot.right(0.5)
            cv2.putText(depth_colormap, f'AGGRESSIVE: {intensity} RIGHT {distance:.0f}mm', 
                       (100, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        else:
            robot.forward(speed)
            cv2.putText(depth_colormap, f'AGGRESSIVE: {intensity} {distance:.0f}mm!', 
                       (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        print(f'Aggression: {intensity} {distance:.0f}mm (speed: {speed})', end='\r')
    else:

        dog_movement['active'] = False
        dog_movement['barking'] = 0
        # No human detected - spin to search
        robot.spinRight(0.3)
        cv2.putText(depth_colormap, 'AGGRESSIVE: Searching for target...', 
                   (130, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        print('Aggression: Checking for target', end='\r')

def curious_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """
    Curious behavior - cautious investigation with circling
    
    Behavior pattern:
    - Far (>800mm): Approach with tracking
    - Medium (400-800mm): Slow approach
    - Close (<400mm): Circle around to investigate
    - None detected: Explore environment
    
    Circling achieved by combining forward movement with rotation
    """

    if human_detected and distance != float('inf'):
        # Draw cyan curiosity indicator
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (255, 255, 0), 2)
        
        # Calculate position for tracking
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2
        
        if distance < 400:
            # Close range - circle around to investigate
            robot.forward(0.2)  # Move forward slowly
            robot.right(0.3)    # While turning
            cv2.putText(depth_colormap, f'CURIOUS: Investigating {distance:.0f}mm', 
                       (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        elif distance < 800:
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.2)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.2)
            else:
                robot.forward(0.2)
            cv2.putText(depth_colormap, f'CURIOUS: Approaching {distance:.0f}mm', 
                       (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 100), 2)
        else:
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.3)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.3)
            else:
                robot.forward(0.4)
            cv2.putText(depth_colormap, f'CURIOUS: Interested {distance:.0f}mm', 
                       (140, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 200, 0), 2)
    else:
        # No human detected - explore environment
        robot.forward(0.2)
        robot.right(0.2)
        cv2.putText(depth_colormap, 'CURIOUS: Exploring...', 
                   (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)

**Step 5: Keyboard Control and Main Callback**

Based on Answer_keyboard_control.ipynb pattern for keyboard input handling.

In [7]:
# NEW: Gesture Recognition Handler
class GestureModeHandler:
    """Handles MediaPipe gesture recognition and mode switching"""
    def __init__(self):
        self.current_mode = 'stop'
        self.last_gesture_time = 0
        self.gesture_cooldown = 1.0  # 1 second between gestures
    
    def print_result(self, result, output_image, timestamp_ms):
        global current_behavior
        
        # Cooldown to prevent rapid switching
        current_time = time.time()
        if current_time - self.last_gesture_time < self.gesture_cooldown:
            return
        
        if result.gestures:
            top_gesture = result.gestures[0][0]
            confidence = top_gesture.score
            name = top_gesture.category_name
            
            # Debug: Show detected gestures
            print(f"Detected: {name} (confidence: {confidence:.2f})")
            
            if confidence >= 0.7:  # High confidence threshold
                old_mode = current_behavior
                
                # Map gestures to behaviors
                if name == "Thumb_Up":
                    current_behavior = 'love'
                elif name == "Thumb_Down":
                    current_behavior = 'fear'
                elif name == "Open_Palm":
                    current_behavior = 'curious'
                elif name in ["Victory"]:
                    current_behavior = 'aggressive'
                #elif name == "Victory":
                    #current_behavior = 'stop'
                    robot.stop()
                
                # Only print when mode changes
                if old_mode != current_behavior:
                    self.last_gesture_time = current_time
                    print(f'\n[GESTURE] {name} → {current_behavior.upper()} mode')

# Initialize gesture handler
gesture_handler = GestureModeHandler()

# Create gesture recognizer with model
gesture_options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_path='gesture_recognizer.task'),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=gesture_handler.print_result
)

# Initialize recognizer
gesture_recognizer = GestureRecognizer.create_from_options(gesture_options)

print("✅ Gesture recognition initialized successfully!")

✅ Gesture recognition initialized successfully!


W0000 00:00:1764928169.695426    9530 gesture_recognizer_graph.cc:129] Hand Gesture Recognizer contains CPU only ops. Sets HandGestureRecognizerGraph acceleration to Xnnpack.
I0000 00:00:1764928169.699366    9530 hand_gesture_recognizer_graph.cc:250] Custom gesture classifier is not defined.
Error in cpuinfo: prctl(PR_SVE_GET_VL) failed
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1764928169.740418    9650 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764928169.788518    9651 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764928169.790896    9651 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764928169.791192    9651 inference_feedback_ma

In [8]:
def main_callback(change):
    """
    Main callback function with gesture recognition
    """
    try:
        # Get updated frame
        frame = change['new']
    
        if frame is None:
            return

        # Create depth colormap for display
        depth_colormap = cv2.applyColorMap(
            cv2.convertScaleAbs(camera.depth_image, alpha=0.03), 
            cv2.COLORMAP_JET)
        
        # NEW: Process gesture recognition
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
        timestamp_ms = int(time.time() * 1000)
        gesture_recognizer.recognize_async(mp_image, timestamp_ms)
    
        # Detect humans and measure distance
        human_detected, distance, bbox = detect_human(frame)
    
        # Execute behavior based on current mode
        if current_behavior == 'love':
            love_behavior(frame, depth_colormap, human_detected, distance, bbox)
        elif current_behavior == 'fear':
            fear_behavior(frame, depth_colormap, human_detected, distance, bbox)
        elif current_behavior == 'aggressive':
            aggressive_behavior(frame, depth_colormap, human_detected, distance, bbox)
        elif current_behavior == 'curious':
            curious_behavior(frame, depth_colormap, human_detected, distance, bbox)
        else:  # stop
            robot.stop()
            cv2.putText(depth_colormap, 'STOPPED', 
                   (250, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)
    
        # Display current behavior mode on frame
        cv2.putText(frame, f'Mode: {current_behavior.upper()}', 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
        # Display images
        scale = 0.3
        resized_color = cv2.resize(frame, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
        resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
        display_color.value = bgr8_to_jpeg(resized_color)
        display_depth.value = bgr8_to_jpeg(resized_depth)
    
    except Exception as e:
        print(f"Main callback error: {e}")

camera.observe(main_callback, names=['color_value'])

print('\n=== Gesture-Controlled Robot Active ===')
print('Gesture controls:')
print('  👍 Thumb Up = Love')
print('  👎 Thumb Down = Fear')
print('  ✋ Open Palm = Curious')
print('  ✌️ Victory = Aggressive')
#print('  ✌️ Victory/Peace = Stop')
print('======================================')


=== Gesture-Controlled Robot Active ===
Gesture controls:
  👍 Thumb Up = Love
  👎 Thumb Down = Fear
  ✋ Open Palm = Curious
  ✊ Victory = Aggressive


W0000 00:00:1764928170.034239    9650 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.6.2 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
jax 0.6.2 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
geopandas 1.1.1 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.


[12/05/2025-09:49:40] [TRT] [I] Loaded engine size: 52 MiB
[12/05/2025-09:49:40] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[12/05/2025-09:49:40] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +36, now: CPU 1, GPU 84 (MiB)
Detected: Thumb_Up (confidence: 0.64)
Detected: Thumb_Up (confidence: 0.70)

[GESTURE] Thumb_Up → LOVE mode
Detected: None (confidence: 0.51)
Detected: None (confidence: 0.79)
Detected: None (confidence: 0.65)
Detected: Thumb_Down (confidence: 0.46)
Detected: Thumb_Down (confidence: 0.49)
Detected: Thumb_Down (confidence: 0.71)

[GESTURE] Thumb_Down → FEAR mode
Detected: Thumb_Down (confidence: 0.50)
Detected: Thumb_Down (confidence: 0.83)
Detected: Thumb_Down (confidence: 0.80)
Detected: Thumb_Down (confidence: 0.73)
Detected: None (confidence: 0.61)
Detected: None (confidence: 0.50)
Detected: None (confidence: 0

**To stop the system:**

In [9]:
# stop camera thread and motors
camera.stop()
print('System stopped - camera and robot halted')

System stopped - camera and robot halted


## Implementation Notes

### Distance Calculation
Distance is measured by:
1. Getting bounding box from YOLO: `bbox = result.boxes.xyxy[i]` (format: [x1, y1, x2, y2])
2. Calculating center: `center_x = (bbox[0] + bbox[2]) / 2`, `center_y = (bbox[1] + bbox[3]) / 2`
3. Reading depth at center: `distance = camera.depth_image[center_y, center_x]`
4. Depth image returns distance in millimeters


### Behavior Distance Thresholds:

**Love:** 450mm (too close) | 450-600mm (perfect) | 600-1200mm (medium) | >1200mm (far)

**Fear:** 500mm (danger) | 900mm (caution) | 1300mm (alert) | >1300mm (safe)

**Aggressive:** 300mm (ram) | 600mm (engage) | 1000mm (rush) | >1000mm (chase)

**Curious:** 400mm (investigate) | 800mm (approach) | >800mm (interested)